Install the dataset


In [ ]:
!pip install CocoDataset==0.1.2

we build the data with https://www.tensorflow.org/datasets/api_docs/python/tfds/folder_dataset/ImageFolder, 

tfds.folder_dataset.ImageFolder, needs to have an specific stracture. (check the documentation)


Now, we create the stracture, but befire we need to mount the drive from left side panel-> files -> click mount drive

In [ ]:
!pwd

/content/drive/MyDrive/coco_data/train


mount the drive and select the directory

first, we set the training directory

In [ ]:
cd /content/drive/MyDrive/coco_data/train


/content/drive/MyDrive/coco_data/train


Download the annotations

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip /content/drive/MyDrive/coco_data/train/annotations_trainval2014.zip





--2022-03-14 21:25:11--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.137.193
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.137.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip.2’

annotations_trainva 100%[===================>] 241.16M  41.9MB/s    in 6.0s    

2022-03-14 21:25:17 (40.1 MB/s) - ‘annotations_trainval2014.zip.2’ saved [252872794/252872794]

Archive:  /content/drive/MyDrive/coco_data/train/annotations_trainval2014.zip
  inflating: annotations/instances_train2014.json  
  inflating: annotations/instances_val2014.json  
  inflating: annotations/person_keypoints_train2014.json  
  inflating: annotations/person_keypoints_val2014.json  
  inflating: annotations/captions_train2014.json  
  inflating: annotations/captions_val2014.json  


class names for downloading


In [ ]:
classes = ['airplane', "apple", "banana", "boat", "bus", "car","person","bicycle","bird","book"]

write the function to download the class with annotations and save it to content folder

In [ ]:
from coco_dataset import coco_dataset_download as cocod


def manual_download(names):
  class_name=names  #class name example 
  images_count=500       #count of images  
  annotations_path='/content/drive/MyDrive/coco_data/train/annotations/instances_train2014.json' #path of coco dataset annotations 
  #call download function 
  cocod.coco_dataset_download(class_name,images_count,annotations_path)





In [ ]:
for i in classes:
  manual_download(i)
  

In [ ]:
# for removing the annotations
!rm -rf annotations

Now we set the test directory and download the daata these

In [ ]:
cd /content/drive/MyDrive/coco_data/test

/content/drive/MyDrive/coco_data/test


In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip /content/drive/MyDrive/coco_data/test/annotations_trainval2014.zip





--2022-03-14 21:51:41--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.20.139
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.20.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  36.0MB/s    in 6.5s    

2022-03-14 21:51:48 (37.2 MB/s) - ‘annotations_trainval2014.zip’ saved [252872794/252872794]

Archive:  /content/drive/MyDrive/coco_data/test/annotations_trainval2014.zip
  inflating: annotations/instances_train2014.json  
  inflating: annotations/instances_val2014.json  
  inflating: annotations/person_keypoints_train2014.json  
  inflating: annotations/person_keypoints_val2014.json  
  inflating: annotations/captions_train2014.json  
  inflating: annotations/captions_val2014.json  


In [ ]:
from coco_dataset import coco_dataset_download as cocod


def manual_download(names):
  class_name=names  #class name example 
  images_count=30       #count of images  
  annotations_path='/content/drive/MyDrive/coco_data/test/annotations/instances_val2014.json' #path of coco dataset annotations 
  #call download function 
  cocod.coco_dataset_download(class_name,images_count,annotations_path)


In [ ]:
for i in classes:
  manual_download(i)

In [ ]:
# for removing the annotations
!rm -rf annotations

now we build the data from folder as tensors


In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

builder = tfds.ImageFolder('/content/drive/MyDrive/coco_data/')
print(builder.info)  # num examples, labels... are automatically calculated
dataset = builder.as_dataset(shuffle_files=True, as_supervised = True)
train_ds, test_ds = dataset["train"], dataset["test"]

tfds.core.DatasetInfo(
    name='image_folder',
    version=1.0.0,
    description='Generic image classification dataset.',
    homepage='https://www.tensorflow.org/datasets/catalog/image_folder',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=11),
    }),
    total_num_examples=5320,
    splits={
        'test': 310,
        'train': 5010,
    },
    supervised_keys=('image', 'label'),
    citation="""""",
    redistribution_info=,
)



to preprocess the data pipeline

In [ ]:

import numpy as np


def prepare_coco_data(mnist):
  #flatten the images into vectors
  mnist = mnist.map(lambda img, target: (tf.image.resize(img, [64,64],
                                         method = tf.image.ResizeMethod.BILINEAR, 
                                         preserve_aspect_ratio=False),      target))
  
  #convert data from uint8 to float32
  mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
  #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
  mnist = mnist.map(lambda img, target: ((img/128.)-1., target))
  #create one-hot targets
  mnist = mnist.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  #cache this progress in memory, as there is no need to redo it; it is deterministic after all
  mnist = mnist.cache()
  #shuffle, batch, prefetch
  mnist = mnist.shuffle(1000)
  mnist = mnist.batch(32)
  mnist = mnist.prefetch(20)
  #return preprocessed dataset
  return mnist

train_dataset = train_ds.apply(prepare_coco_data)
test_dataset = test_ds.apply(prepare_coco_data)

build the model(demo, just to test the data)

In [ ]:
from tensorflow.keras.layers import Dense


class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu')
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.drop_1 = tf.keras.layers.Dropout(0.5)

        self.conv2 = tf.keras.layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu')
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.drop_2 = tf.keras.layers.Dropout(0.5)

        self.maxpool = tf.keras.layers.MaxPooling2D(pool_size=2,strides=2,padding='same')
        self.conv3 = tf.keras.layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',activation='relu')
        self.batch_norm_3 = tf.keras.layers.BatchNormalization()
        self.drop_3 = tf.keras.layers.Dropout(0.75)

        self.conv4 = tf.keras.layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',activation='relu')
        self.batch_norm_4 = tf.keras.layers.BatchNormalization()
        self.drop_4 = tf.keras.layers.Dropout(0.75)

        self.maxpool2 = tf.keras.layers.MaxPooling2D(pool_size=2,strides=2,padding='same')
        self.batch_norm_5 = tf.keras.layers.BatchNormalization()
        self.drop_5 = tf.keras.layers.Dropout(0.2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.out = tf.keras.layers.Dense(10, activation=tf.nn.softmax)



    @tf.function
    def call(self, x):
        #print(x.shape)
        x = self.conv1(x)
        #print(x.shape)
        x = self.batch_norm_1(x)
        x = self.drop_1(x)
        x = self.conv2(x)
        #print(x.shape)
        x = self.batch_norm_2(x)
        x = self.drop_2(x)
        x = self.maxpool(x)
        #print(x.shape)
        # x = self.batch_norm_3(x)
        # x = self.drop_3(x)
        # x = self.conv3(x)
        # #print(x.shape)
        # x = self.conv4(x)
        # x = self.batch_norm_4(x)
        # x = self.drop_4(x)
        #print(x.shape)
        x = self.maxpool2(x)
        x = self.batch_norm_5(x)
        x = self.drop_5(x)
        #print(x.shape)
        x = self.avgpool(x)
        #print(x.shape)
        x = self.out(x)
        return x
    

train steps, takem from complete model train file

In [ ]:
def train_step(model, input, target, loss_function, optimizer):
  # loss_object and optimizer_object are instances of respective tensorflow classes
  with tf.GradientTape() as tape:
    prediction = model(input)
    loss = loss_function(target, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

def test(model, test_data, loss_function):
  # test over complete test data

  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (input, target) in test_data:
    prediction = model(input)
    sample_test_loss = loss_function(target, prediction)
    sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
    sample_test_accuracy = np.mean(sample_test_accuracy)
    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

  test_loss = tf.reduce_mean(test_loss_aggregator)
  test_accuracy = tf.reduce_mean(test_accuracy_aggregator)

  return test_loss, test_accuracy

In [ ]:
tf.keras.backend.clear_session()

#For showcasing we only use a subset of the training and test data (generally use all of the available data!)
train_dataset = train_dataset.take(100)
test_dataset = test_dataset.take(10)

### Hyperparameters
num_epochs = 50
learning_rate = 0.001

# Initialize the model.
model = MyModel()
# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer: ADAM with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Initialize lists for later visualization.
train_losses = []

test_losses = []
test_accuracies = []

#testing once before we begin
test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, _ = test(model, train_dataset, cross_entropy_loss)
train_losses.append(train_loss)

# We train for num_epochs epochs.
for epoch in range(num_epochs):
    print(f'Epoch: {str(epoch)} starting with accuracy {test_accuracies[-1]}')

    #training (and checking in with training)
    epoch_loss_agg = []
    for input,target in train_dataset:
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        epoch_loss_agg.append(train_loss)
    
    #track training loss
    train_losses.append(tf.reduce_mean(epoch_loss_agg))

    #testing, so we can track accuracy and test loss
    test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

Epoch: 0 starting with accuracy 0.0934659090909091
Epoch: 1 starting with accuracy 0.23948863636363638
Epoch: 2 starting with accuracy 0.21647727272727274
Epoch: 3 starting with accuracy 0.24602272727272725
Epoch: 4 starting with accuracy 0.24147727272727276
Epoch: 5 starting with accuracy 0.23892045454545455
Epoch: 6 starting with accuracy 0.2690340909090909
Epoch: 7 starting with accuracy 0.25227272727272726
Epoch: 8 starting with accuracy 0.2605113636363636
Epoch: 9 starting with accuracy 0.2792613636363636
Epoch: 10 starting with accuracy 0.30568181818181817
Epoch: 11 starting with accuracy 0.3159090909090909
Epoch: 12 starting with accuracy 0.2667613636363636
Epoch: 13 starting with accuracy 0.2863636363636364
Epoch: 14 starting with accuracy 0.29772727272727273
Epoch: 15 starting with accuracy 0.31732954545454545
Epoch: 16 starting with accuracy 0.3125
Epoch: 17 starting with accuracy 0.31732954545454545
Epoch: 18 starting with accuracy 0.30568181818181817
Epoch: 19 starting with

In [1]:
import matplotlib.pyplot as plt

# Visualize accuracy and loss for training and test data.
plt.figure()
line1, = plt.plot(train_losses)
line2, = plt.plot(test_losses)
line3, = plt.plot(test_accuracies)
plt.xlabel("Training steps")
plt.ylabel("Loss/Accuracy")
plt.legend((line1,line2, line3),("training","test", "test accuracy"))
plt.show()

NameError: ignored

<Figure size 432x288 with 0 Axes>